# Implementation of SHA-1 Core on SoC Ported to Python

In [ ]:
# Internal constants definitions
ADDR_I_CS_DATA      = 0x10
BITS_I_CS_DATA      = 1
ADDR_I_WE_DATA      = 0x18
BITS_I_WE_DATA      = 1
ADDR_I_ADDRESS_DATA = 0x20
BITS_I_ADDRESS_DATA = 8
ADDR_I_WRITE_DATA   = 0x28
BITS_I_WRITE_DATA   = 32
ADDR_O_READ_DATA    = 0x30
BITS_O_READ_DATA    = 32
ADDR_O_READ_CTRL    = 0x34
ADDR_O_ERROR_DATA   = 0x38
BITS_O_ERROR_DATA   = 1
ADDR_O_ERROR_CTRL   = 0x3c

In [ ]:
# Load the overlay containing the IP
from pynq import Overlay

overlay = Overlay('/usr/local/lib/python3.6/dist-packages/sha1_pynq/bitstream/sha1_overlay.bit')

In [ ]:
# See what attributes the overlay has
overlay?

In [ ]:
# Access the the sha1 attribute to create a driver for the IP
sha1_ip = overlay.sha1_control_0
sha1_ip?

In [ ]:
def check_core():
    '''Print the name of the IP core and the version.
    
    Arguments:
    None
    
    Return:
    None
    '''
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name0 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name0)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name1 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name1)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 2)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    version = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(version)[2:]).decode())

In [ ]:
def read_word(address):
    '''Read a data word from the given address
    
    Arguments:
    address - the address to read data from
    
    Return:
    The data that was read from memory
    '''
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    word =0;
    while(sha1_ip.read(ADDR_O_READ_DATA) != 0):
        word = sha1_ip.read(ADDR_O_READ_DATA)
        break
    return word

In [ ]:
def read_digest():
    '''Read the digest of the message that was input
    
    Arguments:
    None
    
    Return:
    A byte array of length 20 containing the digest data
    '''
    
    digest = bytearray()
    for i in range(5):
        data = read_word(0x20 + i)
        digest.extend(data.to_bytes(4, byteorder='big'))
    return digest

In [ ]:
def write_word(address, word):
    '''Write a data word to the given address
    
    Arguments:
    address - the address to write to
    word - the data to write
    
    Return:
    None
    '''
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_WRITE_DATA, word)

In [ ]:
def pad_message(msg):
    '''Pad the input message following the SHA1 documentation.
    The purpose of the padding is to make the padded message have 
    a total length that is a multiple of 512, which can be processed
    as sequential blocks of 512 bits.
    
    Arguments:
    msg - input message as an ascii string
    
    Return:
    A byte array containing the bytes of the padded message
    '''
    
    blocks = bytearray(msg, 'ascii')
    msgSize = len(msg)
    blocks.append(0x80)
    while len(blocks) % 64 != 56:
        blocks.append(0x00)
    reserved = (msgSize * 8).to_bytes(8, byteorder='big')
    blocks.extend(reserved)
    return bytes(blocks)

In [ ]:
def write_block(block, ctrl_val):
    '''Write a block of the padded message.
    
    Arguments:
    block - 512 bits of data to be written
    ctrl_val - control value based on what block is being written
    
    Return:
    None
    '''
    
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 1)
    j = 0
    for i in range(16):
        word = int.from_bytes(block[j:j+4], byteorder='big')
        write_word(0x10 + i, word)
        j += 4
    write_word(0x8, ctrl_val)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)

In [ ]:
def write_blocks(blocks):
    '''Write each block of the padded message.
    
    Arguments:
    blocks - A byte array containing the bytes of the padded message
    
    Return:
    None
    '''
    
    bytesWritten = 0
    while bytesWritten < len(blocks):
        if bytesWritten == 0:
            write_block(blocks[bytesWritten:bytesWritten + 64], 0x1)
        else:
            write_block(blocks[bytesWritten:bytesWritten + 64], 0x2)
        
        status = read_word(0x09)
        while status == 0:
            status = read_word(0x09)
            print("status:", status)
        
        bytesWritten += 64

In [ ]:
def hw_digest(msg):
    '''Pad the inpute message, write the sequential blocks, and read the digest of the input.
    
    Arguements:
    msg - input message as an ascii string
    
    Return:
    A byte array of length 20 containing the digest data
    '''
    
    msgPadded = pad_message(msg)
    write_blocks(msgPadded)
    digest = read_digest()
    return digest

In [ ]:
import hashlib
def sw_digest(msg):
    '''Get the SHA1 digest of an ascii message using internal Python library.
    
    Arguments:
    msg - input message as an ascii string
    
    Return:
    A SHA1 hash object of the digest for the input message
    '''
    
    digest = hashlib.sha1(msg.encode())
    return digest

In [ ]:
def test_sha1(msg):
    '''Check if the message digests from hardware and software are the same.
    
    Arguments:
    msg - input message as an ascii string
    
    Return:
    None
    '''
    
    swDigest = sw_digest(msg).hexdigest()
    hwDigest = hw_digest(msg).hex()
    print("hw digest: {}".format(hwDigest))
    print("sw digest: {}".format(swDigest))
    
    if swDigest == hwDigest:
        print("Test Passed!")
    else:
        print("Test Failed!")

In [ ]:
# Test 1
print("----------------------------------------")
print("Single Block Test")
print("----------------------------------------")
test_sha1("abc")
print("\n----------------------------------------")
print("Double Block Test")
print("----------------------------------------")
test_sha1("abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq")

In [ ]:
# Test 2
test_sha1("Hello world")

In [ ]:
# Test 3
test_sha1("Reconfigurable GPU Computing Lab")

In [ ]:
# Custom message test
m = input("Enter a message to hash with SHA1: ")
test_sha1(m)

In [ ]:
%%timeit
# Software Timing
sw_digest("Hello World!")

In [ ]:
%%timeit
# Hardware Timing
hw_digest("Hello World!")